# GitHub Data for Open Services Initiatives

In this notebook we will quickly walk through how to access and start analyzing the Issue/PR data for all the sub-projects under different SIGs of the Open Services Group. We also explore the various fields in the Issue/PR dataset and the features that we can derive from them in order to calculate metrics to measure and track the progress and contribution made across the Open Services group.

## Data Collection

The Thoth team already developed a tool for fetching and collecting data from GitHub repositories, [MI tool](https://github.com/thoth-station/mi). Using this tool, we will fetch the GitHub issue/PR data for all the repos mentioned in [sigs.yaml](https://github.com/open-services-group/community/blob/main/sigs.yaml) and store them to Ceph. We will then access the data from Ceph into the notebook and explore the various fields in the data that can be used to derive metrics. We would like to automate this data collection in the future, so that the notebook can be run daily on the most recent data for which we plan to integrate the [MI scheduler](https://github.com/thoth-station/mi-scheduler) tool into our metrics processing pipeline.

In [1]:
import os
import boto3
import json
import pandas as pd
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
## CEPH Bucket variables
## Create a .env file on your local with the correct configs,
s3_endpoint_url = os.getenv("S3_ENDPOINT")
s3_access_key = os.getenv("S3_ACCESS_KEY")
s3_secret_key = os.getenv("S3_SECRET_KEY")
s3_bucket = os.getenv("S3_BUCKET")
s3_path = os.getenv("S3_PROJECT_KEY", "open-services-group/metrics/github")
s3_input_data_path = "raw_data"
REMOTE = os.getenv("REMOTE")

In [3]:
s3 = boto3.resource(
    "s3",
    endpoint_url=s3_endpoint_url,
    aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_secret_key,
)

In [4]:
# Function to read data from Ceph
def read_data_from_ceph(s3_file_path):
    print("Getting dataset from Ceph")
    content = s3.Object(s3_bucket, s3_file_path)
    file = content.get()["Body"].read().decode("utf-8")
    data = json.loads(file)
    separated_list = data.splitlines()
    data_dict = {
        str(i): json.loads(separated_list[i]) for i in range(0, len(separated_list))
    }
    df = pd.DataFrame(data_dict).T
    return df

Lets now fetch the PR/Issue data for all the sub-projecs defined under the following SIGs/WGs:

* **SIG Data Science** comprises of the following sub-projects:
  * [OS Climate](https://github.com/open-services-group/community/blob/main/sigs.yaml#L93)
  * [Operate First Data Science Community](https://github.com/aicoe-aiops/operate-first-data-science-community)
  * [Metrics](https://github.com/open-services-group/metrics)
        
* **SIG Community Experience** comprises of the following sub-projects:
  * [ComCom](https://github.com/open-services-group/community/main)
  * [GovOrg](https://github.com/open-services-group/community/main)
  * [Metrics Strategic](https://github.com/open-services-group/community/main)
       
* **SIG Services** comprises of the following sub-projects:
  * [AICoE-CD](https://github.com/AICoE/aicoe-cd)
  * [AICoE-CI](https://github.com/AICoE/aicoe-ci)
  * Project Meteor
      * https://github.com/AICoE/meteor
      * https://github.com/AICoE/meteor-operator
  * [Project Thoth](https://github.com/thoth-station/core)
  * [Seraph](https://github.com/AICoE/seraph)
       
* **WG BYON Build Pipelines** comprises of the following sub-project:
  * [BYON](https://github.com/open-services-group/byon)
       
* **[WG DevSecOps](https://github.com/open-services-group/devsecops)**

## SIG Data Science

The Data Science SIG comprises of 3 sub-projects.

Lets look at the sub-project `OS Climate` and fetch all its relevant Issue/PR data

In [5]:
os_climate_issues = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/os-climate/aicoe-osc-demo/Issue.json"
)
os_climate_prs = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/os-climate/aicoe-osc-demo/PullRequest.json"
)

Getting dataset from Ceph
Getting dataset from Ceph


In [6]:
os_climate_issues.head()

,title,body,created_by,created_at,closed_by,closed_at,labels,interactions,id
0,Prepare GPU image for training teacher network...,**Is your feature request related to a problem...,pacospace,1644527768,None,None,"{'enhancement': {'color': 'a2eeef', 'labeled_a...",{'pacospace': 8},128
1,Value Error related to S3 in demo2 notebook,Value Error related to S3 at Import in when ru...,andraNew,1641590227,andraNew,1642091274,"{'bug': {'color': 'd73a4a', 'labeled_at': 1641...","{'andraNew': 123, 'erikerlandson': 90, 'chauha...",127
2,Create Jupyterbook,Add _toc.yaml and _config.yml for the repo and...,oindrillac,1639764581,oindrillac,1640024593,{},{},125
3,Should we state which base image tag is used i...,See: https://github.com/os-climate/aicoe-osc-d...,pacospace,1639086094,MichaelClifford,1639189607,{},"{'chauhankaranraj': 71, 'pacospace': 39}",122
4,Use overlays for building images,**Is your feature request related to a problem...,chauhankaranraj,1639077221,None,None,"{'enhancement': {'color': 'a2eeef', 'labeled_a...","{'pacospace': 19, 'chauhankaranraj': 40}",121


In [7]:
os_climate_prs.head()

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
0,Add manifests for GPU image build,Signed-off-by: Francesco Murdaca <fmurdaca@red...,XL,pacospace,1644855027,None,None,None,None,1,9,{'pacospace': 22},"{'881638114': {'author': 'erikerlandson', 'wor...",[],[d2af49daf938560856f315e573cd9e54e92aa570],"[manifests/.sops.yaml, manifests/README.md, ma...",1644863765,1644863765,129
1,Updated documentation,closes #125 \r\ncloses #110 \r\n\r\nJupyterBoo...,M,oindrillac,1639777200,1640024594,oindrillac,1640024593,oindrillac,1,5,"{'oindrillac': 13, 'chauhankaranraj': 18}","{'835457435': {'author': 'aakankshaduggal', 'w...",[],[523e26606333956764986a296149ca56edf56b40],"[README.md, _config.yml, _toc.yml, notebooks/d...",1639778743,1639793939,126
2,Update README,This PR \r\n- updates the README to mention th...,XS,chauhankaranraj,1639532708,1639591055,MichaelClifford,1639591055,MichaelClifford,1,1,{'MichaelClifford': 1},{},[],[ad9668f096e1e5ebc5b123d1c6df4ea5ed98af5a],[notebooks/demo2/README.md],None,None,124
3,Use specific versions instead of latest for im...,Closes #122,S,chauhankaranraj,1639094750,1639189607,MichaelClifford,1639189607,MichaelClifford,1,3,{'MichaelClifford': 1},"{'828089083': {'author': 'oindrillac', 'words_...",[],[c0666de738d99cf9a40e2867fc8f4e51a305d086],"[Dockerfile, notebooks/demo2/inference.pipelin...",1639094897,1639094897,123
4,Finalize Demo 2 readme,This PR adds Superset dashboard link to the Re...,XS,Shreyanand,1638982645,1639079020,MichaelClifford,1639079020,MichaelClifford,1,1,{'MichaelClifford': 1},"{'827649205': {'author': 'chauhankaranraj', 'w...",[],[ce265997ec1e5bf368d804f571e660edb261a77a],[notebooks/demo2/README.md],1639072873,1639072873,119


Lets look at the sub-project `Operate First Data Science Community` and fetch all its relevant Issue/PR data

In [8]:
opf_ds_community_issues = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/aicoe-aiops/"
    "operate-first-data-science-community/Issue.json"
)
opf_ds_community_prs = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/aicoe-aiops/"
    "operate-first-data-science-community/PullRequest.json"
)

Getting dataset from Ceph
Getting dataset from Ceph


In [9]:
opf_ds_community_issues.head()

,title,body,created_by,created_at,closed_by,closed_at,labels,interactions,id
0,[Metric] - Create a visualization Dashboard,Subissue for #35 \r\n\r\n- [ ] Get started wit...,aakankshaduggal,1645050878,None,None,{},{},45
1,Operate First Data Science Community Meetup #7,## Please provide the following info about you...,aakankshaduggal,1644528835,None,None,{},{},42
2,update calendar invite,None,MichaelClifford,1644445287,MichaelClifford,1644454274,{},{},41
3,reach out to external speakers,None,MichaelClifford,1644445029,None,None,{},{},40
4,[Metric] Collect engagement metrics on tweets ...,We want to view metrics such as Likes/Retweets...,oindrillac,1644371111,None,None,{},{},38


In [10]:
opf_ds_community_prs.head()

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
0,Add actionable items to the issue_template,Resolve #29,M,aakankshaduggal,1644977887,1645053971,sesheta,1645053971,sesheta,1,2,"{'MichaelClifford': 62, 'aakankshaduggal': 68,...","{'883681332': {'author': 'oindrillac', 'words_...","[size/M, approved]",[f85849656ec1b8dba052514b97675d9e366eb926],"[.github/ISSUE_TEMPLATE/event_template.md, .gi...",1644979349,1644980004,44
1,Update Agenda for meetup 6,None,S,aakankshaduggal,1644536203,1644538630,sesheta,1644538630,sesheta,1,1,"{'MichaelClifford': 13, 'sesheta': 65}","{'879321299': {'author': 'suppathak', 'words_c...","[size/S, approved, lgtm]",[e166675a1c01c9072f1aaf01d08069e2f05ab019],[docs/content.md],1644538067,None,43
2,Add the privacy policy for the Opf DS Communit...,This PR resolves #28,M,aakankshaduggal,1644439870,None,None,None,None,1,1,"{'sesheta': 73, 'aakankshaduggal': 75, 'eriker...","{'877832604': {'author': 'suppathak', 'words_c...","[size/M, lgtm]",[b6b707fa49d80d99bad8823083eabab424f6962d],[docs/privacy-policy.md],1644446751,None,39
3,Added a notebook for exploring data obfuscatio...,"In this notebook, we explored an obfuscation t...",XXL,suppathak,1644370123,1645053734,sesheta,1645053733,sesheta,1,1,"{'review-notebook-app[bot]': 29, 'MichaelCliff...","{'877691435': {'author': 'oindrillac', 'words_...","[approved, size/XXL]",[4fac57b106ef08df2dd73ea1e875887f7da4ef6e],[notebooks/data_obfuscation.ipynb],1644440520,None,33
4,Add sample attendees data,This PR adds a sample data for #27,XL,aakankshaduggal,1644004481,1644346518,sesheta,1644346518,sesheta,1,2,"{'MichaelClifford': 25, 'aakankshaduggal': 87,...","{'873193483': {'author': 'oindrillac', 'words_...","[approved, size/XL]",[92202e4bd6330edc99659b3794011f555c168735],"[data/raw/sample_data.csv, notebooks/create_da...",1644004656,1644004656,32


Lets look at the sub-project `Metrics` and fetch all its relevant Issue/PR data

In [11]:
metric_issues = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/open-services-group/metrics/Issue.json"
)
metric_prs = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/open-services-group/metrics/PullRequest.json"
)

Getting dataset from Ceph
Getting dataset from Ceph


In [12]:
metric_issues.head()

,title,body,created_by,created_at,closed_by,closed_at,labels,interactions,id
0,Collect metrics for all sub-projects in the OS...,Collect GitHub data using MI scheduler i.e iss...,hemajv,1644955949,None,None,{},"{'oindrillac': 78, 'hemajv': 102}",13
1,Spike: Define OKR completion,This issue is created to discuss different app...,Shreyanand,1644275165,None,None,{},"{'MichaelClifford': 33, 'Shreyanand': 24, 'hem...",10
2,Project workflow diagram,Create an architecture diagram describing the ...,hemajv,1644274319,hemajv,1644974040,{},"{'oindrillac': 30, 'hemajv': 39, 'MichaelCliff...",9
3,Update Readme,"Update the Readme to reflect project goals, re...",Shreyanand,1644272140,sesheta,1644973068,{},{},8
4,[EPIC] Visualization,Create dashboards/reports to visualize metrics...,hemajv,1643934285,None,None,{},{},7


In [13]:
metric_prs.head()

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
0,Automatic update of dependencies by Kebechet f...,Kebechet has updated the dependencies to the l...,S,khebhut[bot],1644974885,None,None,None,None,1,1,{'sesheta': 202},{},"[size/S, needs-ok-to-test, bot]",[5848f25cc267af83c7ac8267b738f3884f4c5745],[Pipfile.lock],None,None,15
1,Update README,Resolves #8,M,hemajv,1644962453,1644973068,sesheta,1644973068,sesheta,1,2,{'sesheta': 65},"{'883344363': {'author': 'oindrillac', 'words_...","[lgtm, size/M, approved]",[09a63cf6c1b8d058d14c3ca831be672232c4b91e],"[README.md, docs/imgs/project_architecture.jpeg]",1644962849,1644973002,14
2,Update OWNERS,cc @Shreyanand,S,hemajv,1644448066,1644872377,sesheta,1644872377,sesheta,1,1,"{'MichaelClifford': 11, 'sesheta': 65, 'harsha...",{},"[size/S, approved]",[6dff26159faa30e67808db422db9131fb67888e6],[OWNERS],None,None,12
3,Automatic update of dependencies by Kebechet f...,Kebechet has updated the dependencies to the l...,L,khebhut[bot],1644433646,1644974742,sesheta,1644974742,sesheta,1,1,"{'sesheta': 182, 'goern': 1, 'khebhut[bot]': 1...",{},"[size/L, approved, bot, ok-to-test]",[6de8ba9b62f3414c02442605a5445a6401bf5e1b],[Pipfile.lock],None,None,11


## SIG Community Experience

This SIG comprises of 3 sub-projects ComCom, GovOrg and Metrics Strategic that are all residing in a single repository i.e `open-services-group/community`. Let's fetch all the Issue/PRs for this repo.

In [14]:
sig_community_issues = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/open-services-group/community/Issue.json"
)
sig_community_prs = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/open-services-group/community/PullRequest.json"
)

Getting dataset from Ceph
Getting dataset from Ceph


In [15]:
sig_community_issues.head()

,title,body,created_by,created_at,closed_by,closed_at,labels,interactions,id
0,[SPGovOrg] ADR Topics,Collecting and sorting ADR Topics,schwesig,1644962887,None,None,{},{'schwesig': 234},69
1,charter.md Clarify the relationship with the M...,Metrics Strategic vs Metrics\n[Metrics](https:...,schwesig,1644601990,None,None,{},{},66
2,Define 5+ core KPIs,None,schwesig,1644599725,None,None,{'priority/critical-urgent': {'color': 'e11d21...,"{'schwesig': 3, 'sesheta': 47}",65
3,Setup Regular Meeting SP GovOrg,- [ ] finding a time slot\n- [ ] add to google...,schwesig,1644599149,None,None,{'priority/critical-urgent': {'color': 'e11d21...,{'schwesig': 4},64
4,Setup Regular Meeting SP ComCom,- [ ] finding a time slot\n- [ ] add to goog...,schwesig,1644599139,None,None,{'priority/critical-urgent': {'color': 'e11d21...,{'schwesig': 5},63


In [16]:
sig_community_prs.head()

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
0,Added details to WG BYON Readme,Added some additional details to the BYON WG R...,XS,oindrillac,1644882106,1645044002,sesheta,1645044002,sesheta,1,2,{'sesheta': 65},"{'882603479': {'author': 'pacospace', 'words_c...","[lgtm, approved, size/XS]",[4ced068b7d3d598f2625a0fa9365f7132eff72f6],"[sigs.yaml, wg-byon-build-pipelines/README.md]",1644930772,1645043615,68
1,Change ds-sig meeting time from 10 to 11,SSIA :),XS,MichaelClifford,1644873710,1644947372,sesheta,1644947372,sesheta,1,3,"{'MichaelClifford': 4, 'sesheta': 74}","{'881889567': {'author': 'pacospace', 'words_c...","[lgtm, approved, size/XS]",[a7269770e909b22a77a195deace410a183566dcd],"[sig-data-science/README.md, sig-list.md, sigs...",1644874267,1644874267,67
2,Fix a couple of recent changes,The update to `sig-list.md` from [#52](https:/...,XS,codificat,1644597321,1644603432,sesheta,1644603432,sesheta,2,2,{'sesheta': 65},"{'880160348': {'author': 'schwesig', 'words_co...","[lgtm, approved, size/XS]","[12512a57075369b8151cae730e9daa7cfadbf62b, c50...","[sig-list.md, sigs.yaml]",1644603267,1644603267,57
3,Small fix to readme,small fix needed which was missed in #51 \r\n\...,XS,hemajv,1644532913,1644537555,sesheta,1644537555,sesheta,1,1,"{'hemajv': 17, 'oindrillac': 3, 'schwesig': 1,...","{'879240679': {'author': 'oindrillac', 'words_...","[lgtm, approved, size/XS]",[3cf0ede6e9e7ad6a7d56447b8a14c59a9834a9a4],[sig-data-science/README.md],1644534508,None,55
4,Add Samuel to OSG community,@tumido,XS,SamoKopecky,1644504264,1644509713,sesheta,1644509713,sesheta,1,1,{'sesheta': 182},"{'878683172': {'author': 'tumido', 'words_coun...","[lgtm, approved, size/XS, ok-to-test]",[f948fec4f5fa97dfc1be0184ebef70627241bf33],[github-config.yaml],1644509507,1644509507,54


## SIG Services
The SIG Services comprises of 5 sub-projects. Let's look at the `AICoE CD` project repo and fetch its relevant issue/PR data.

In [17]:
aicoe_cd_issues = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/AICoE/aicoe-cd/Issue.json"
)
aicoe_cd_prs = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/AICoE/aicoe-cd/PullRequest.json"
)

Getting dataset from Ceph
Getting dataset from Ceph


In [18]:
aicoe_cd_issues.head()

,title,body,created_by,created_at,closed_by,closed_at,labels,interactions,id
0,stage-thoth-prescriptions-refresh-job app is n...,**Describe the bug**\r\nstage-thoth-prescripti...,pacospace,1642447357,None,None,{},{},746
1,No requirements file found for runtime environ...,Please create requirements file for environmen...,khebhut[bot],1634064023,harshad16,1634066165,"{'bot': {'color': '698b69', 'labeled_at': 1634...",{},708
2,No requirements file found for runtime environ...,Please create requirements file for environmen...,khebhut[bot],1634063990,harshad16,1634066165,"{'bot': {'color': '698b69', 'labeled_at': 1634...",{},707
3,No requirements file found for runtime environ...,Please create requirements file for environmen...,khebhut[bot],1634062726,harshad16,1634066166,"{'bot': {'color': '698b69', 'labeled_at': 1634...",{},706
4,No requirements file found for runtime environ...,Please create requirements file for environmen...,khebhut[bot],1634062695,harshad16,1634066166,"{'bot': {'color': '698b69', 'labeled_at': 1634...",{},705


In [19]:
aicoe_cd_prs.head()

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
0,Remove application definition for dh-prod-rsyslog,## This Pull Request implements\r\n\r\nExplain...,S,accorvin,1644950829,1644951115,accorvin,1644951115,accorvin,1,3,"{'sesheta': 73, 'rimolive': 1}","{'883011542': {'author': 'anishasthana', 'word...","[size/S, lgtm]",[8125f2d7569ce5fec7df21ae5e77cc4d524aeff7],[manifests/overlays/prod/applications/data_hub...,1644951037,None,763
1,Add application definition for s3 webserver,## This Pull Request implements\r\n\r\nExplain...,S,accorvin,1644862075,1644862302,sesheta,1644862302,sesheta,1,3,"{'rimolive': 1, 'sesheta': 65}","{'881605021': {'author': 'lucferbux', 'words_c...","[approved, size/S, lgtm]",[6f87bec013e91afde417139f978d3dd4b23f3006],[manifests/overlays/prod/applications/data_hub...,1644862206,1644862206,762
2,Add argo-workflows App for OCP4 cluster,## This Pull Request implements\r\n\r\nExplain...,S,gmfrasca,1644544614,1644545000,sesheta,1644545000,sesheta,1,1,"{'rimolive': 1, 'sesheta': 74, 'accorvin': 1}",{},"[approved, size/S]",[70c32ee30528e9f7f79cbfb51815ab144e4d9735],[manifests/overlays/prod/applications/data_hub...,None,None,761
3,Added workflow_dispatch to actions script,## This Pull Request implements\r\n\r\nAdded w...,XS,lucferbux,1644353240,1644353573,lucferbux,1644353573,lucferbux,1,1,{'sesheta': 65},"{'876250434': {'author': 'HumairAK', 'words_co...","[size/XS, approved, lgtm]",[e4590eb7e2d6ed1cc72e0768492b365933d38371],[.github/workflows/ci.yaml],1644353446,1644353446,760
4,Added github actions to automate pages deployment,## This Pull Request implements\r\n\r\nAdded G...,S,lucferbux,1644347723,1644348674,sesheta,1644348674,sesheta,1,1,"{'accorvin': 1, 'sesheta': 74}","{'876094785': {'author': 'anishasthana', 'word...","[approved, size/S]",[645ee47bea4847da55e2f8262f42f2164bb2dfc8],[.github/workflows/ci.yaml],1644347833,1644348165,759


Let's look at the `AICoE CI` project repo and fetch its relevant issue/PR data.

In [20]:
aicoe_ci_issues = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/AICoE/aicoe-ci/Issue.json"
)
aicoe_ci_prs = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/AICoE/aicoe-ci/PullRequest.json"
)

Getting dataset from Ceph
Getting dataset from Ceph


In [21]:
aicoe_ci_issues.head()

,title,body,created_by,created_at,closed_by,closed_at,labels,interactions,id
0,Help with Sesheta invite,### GitHub Repo\n\nhttps://github.com/os-clima...,negillett,1644529527,harshad16,1644871770,"{'bot': {'color': '698b69', 'labeled_at': 1644...","{'goern': 14, 'negillett': 75, 'harshad16': 70}",160
1,Pull images from private registries,**Is your feature request related to a problem...,samuelvl,1644009860,None,None,"{'kind/feature': {'color': 'c7def8', 'labeled_...",{'anishasthana': 3},159
2,Help with Sesheta invite,### GitHub Repo\n\nhttps://github.com/theckang...,theckang,1641974866,harshad16,1642020740,"{'bot': {'color': '698b69', 'labeled_at': 1641...",{'harshad16': 5},155
3,Create a general documentation for Thoth CI/CD...,**Is your feature request related to a problem...,mayaCostantini,1637857637,None,None,"{'documentation': {'color': '0075ca', 'labeled...","{'codificat': 15, 'sesheta': 59, 'mayaCostanti...",153
4,Add Tekton Chains to AICoE-CI deployment on op1st,chains is pretty easy to add:\r\n\r\n`kubectl ...,goern,1635173641,None,None,"{'kind/feature': {'color': 'c7def8', 'labeled_...","{'harshad16': 105, 'sesheta': 38, 'codificat':...",150


In [22]:
aicoe_ci_prs.head()

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
0,:city_sunrise: Allow arch build with overlays ...,Allow arch build with overlays feature\r\nSign...,L,harshad16,1644897491,1644897704,sesheta,1644897704,sesheta,1,4,"{'harshad16': 5, 'sesheta': 65}",{},"[approved, size/L]",[1a503b352dd5a23b7bd24457b6f568c7f47da29f],"[pipeline/overlays-release-pipeline.yaml, task...",None,None,161
1,Clean up unused Tasks,## This introduces a breaking change\r\n\r\nHo...,XL,codificat,1643169805,None,None,None,None,1,11,{'sesheta': 73},{},[size/XL],[0341f01b10276a2628a93654f41ece5bccdb78cd],"[kustomization.yaml, setup-instance/kustomizat...",None,None,158
2,Using the python -m build for generating packa...,Using the python -m build for generating packa...,S,harshad16,1642563082,1642563182,harshad16,1642563182,harshad16,1,3,{'sesheta': 73},{},[size/S],[0dcf2a83ab5bb9db8a106801d19c7581a2546bad],"[.prow.yaml, tasks/issue-pypi-release-task.yam...",None,None,157
3,Add resources for container build steps,## Related Issues and Dependencies\r\n\r\nFixe...,S,codificat,1642200205,1642200806,harshad16,1642200806,harshad16,1,2,{'sesheta': 65},"{'853201587': {'author': 'harshad16', 'words_c...","[size/S, approved]",[4624e4e8eb675908e1fb64605ea11a1c5b000f82],"[tasks/issue-release-task.yaml, tasks/tag-buil...",1642200792,1642200792,156
4,Propogate requirements info to a specific dir,Propogate requirements info to a specific dir\...,M,harshad16,1639508188,1639573215,harshad16,1639573215,harshad16,1,5,{'sesheta': 79},"{'831991575': {'author': 'fridex', 'words_coun...",[size/M],[84f265a166c0fdb87d8b3171f1735b00445e94b7],"[docs/gather-metrics-deployments-pipeline.md, ...",1639530919,1639530919,154


Let's look at the `Meteor` project repos and fetch its relevant issue/PR data.

In [23]:
meteor_issues = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/AICoE/meteor/Issue.json"
)
meteor_prs = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/AICoE/meteor/PullRequest.json"
)
meteor_operator_issues = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/AICoE/meteor-operator/Issue.json"
)
meteor_operator_prs = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/AICoE/meteor-operator/PullRequest.json"
)

Getting dataset from Ceph
Getting dataset from Ceph
Getting dataset from Ceph
Getting dataset from Ceph


In [24]:
meteor_issues.head()

,title,body,created_by,created_at,closed_by,closed_at,labels,interactions,id
0,jupyterbook and jupyterhub pipelines build fails,```\r\nSTEP-BUILD\r\n\r\n+ mkdir -p .tekton_me...,pacospace,1638293738,tumido,1638300197,{'priority/critical-urgent': {'color': 'e11d21...,{'tumido': 36},233
1,when inserting URL to meteor shower and click ...,**Describe the bug**\r\nwhen inserting URL to ...,pacospace,1638280403,pacospace,1642181665,{'priority/critical-urgent': {'color': 'e11d21...,"{'tumido': 58, 'pacospace': 29}",230
2,Load the list of pipelines from `Shower` resou...,Follow up to https://github.com/AICoE/meteor-o...,tumido,1638240205,None,None,{},{},228
3,"""no space left on device"" error while building...",While using https://shower.meteor.zone/ to bui...,MichaelClifford,1636157830,tumido,1638242749,"{'kind/bug': {'color': 'e11d21', 'labeled_at':...","{'MichaelClifford': 4, 'tumido': 83, 'goern': 3}",207
4,meteor pipeline for JH image is not copying th...,1. Go to https://shower.meteor.zone/\r\n2. Ins...,pacospace,1635944256,None,None,"{'lifecycle/stale': {'color': '795548', 'label...","{'tumido': 25, 'pacospace': 39, 'sesheta': 38}",200


In [25]:
meteor_operator_issues.head()

,title,body,created_by,created_at,closed_by,closed_at,labels,interactions,id
0,"Add other useful fields. apiVersion, kind, uid?'",https://github.com/AICoE/meteor-operator/blob/...,todo[bot],1638280656,tumido,1638281102,"{'todo :spiral_notepad:': {'color': '00B0D8', ...",{'tumido': 16},58
1,"Add other useful fields. apiVersion, kind, uid?'",https://github.com/AICoE/meteor-operator/blob/...,todo[bot],1638280653,tumido,1638281095,"{'todo :spiral_notepad:': {'color': '00B0D8', ...",{'tumido': 16},57
2,Configure button labels in the pipeline resour...,Follow up after https://github.com/AICoE/meteo...,tumido,1638244936,None,None,{},{},56
3,Reconcile Comas dynamically,Currently a list of namespaces is hardcoded to...,tumido,1638244018,sesheta,1639686267,{},{},55
4,Allow selective on/off of individual pipelines...,Follow up after https://github.com/AICoE/meteo...,tumido,1638240006,None,None,{},{},53


In [26]:
meteor_prs.head()

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
0,chore(deps-dev): Bump eslint-config-next from ...,Bumps [eslint-config-next](https://github.com/...,M,dependabot[bot],1643746681,None,None,None,None,1,2,{'sesheta': 202},{},"[size/M, needs-ok-to-test, ok-to-test]",[bc37f3bb7186408ae9d3d9a8e1fd2ede7efc4692],"[package-lock.json, package.json]",None,None,276
1,chore(deps): Bump next from 12.0.8 to 12.0.10,Bumps [next](https://github.com/vercel/next.js...,L,dependabot[bot],1643746664,None,None,None,None,1,2,{'sesheta': 202},{},"[size/L, needs-ok-to-test, ok-to-test]",[7a6776efd2b2b2182d501a4c6d475fe59fdc8efd],"[package-lock.json, package.json]",None,None,275
2,chore(deps): Bump @patternfly/patternfly from ...,Bumps [@patternfly/patternfly](https://github....,XS,dependabot[bot],1643660628,None,None,None,None,1,2,{'sesheta': 202},{},"[size/XS, needs-ok-to-test, ok-to-test]",[3985f557fa0d88567d304fbec5589dfdae4c39e6],"[package-lock.json, package.json]",None,None,274
3,chore(deps): Bump @patternfly/react-core from ...,Bumps [@patternfly/react-core](https://github....,M,dependabot[bot],1643660586,None,None,None,None,1,2,{'sesheta': 202},{},"[size/M, needs-ok-to-test, ok-to-test]",[d00a255277b8dd9f8cd72f34a2d5a97c5d8a8fb0],"[package-lock.json, package.json]",None,None,273
4,chore(deps): Bump next from 12.0.8 to 12.0.9,Bumps [next](https://github.com/vercel/next.js...,L,dependabot[bot],1643315705,1643746667,dependabot[bot],None,None,1,2,"{'sesheta': 191, 'dependabot[bot]': 3}","{'867784089': {'author': 'tumido', 'words_coun...","[size/L, approved, ok-to-test]",[74708327e609ffec1ce2ce829fb8cb8924cbdd05],"[package-lock.json, package.json]",1643650467,1643650467,272


In [27]:
meteor_operator_prs.head()

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
0,feat: Use externalServices for Comas,Resolves: https://github.com/AICoE/meteor-oper...,S,tumido,1639435496,1639686267,sesheta,1639686267,sesheta,1,2,"{'tumido': 1, 'sesheta': 65}",{},"[approved, size/S, lgtm]",[8ee640e968f1d0a58f278a84fbfb963c23d73ef5],"[api/v1alpha1/zz_generated.deepcopy.go, contro...",None,None,62
1,Additional improvements of the OCP UI for Meteor,Mainly cosmetic changes for more interactive a...,L,tumido,1639432383,1639434108,sesheta,1639434108,sesheta,5,24,"{'tumido': 1, 'sesheta': 65}",{},"[approved, size/L, lgtm]","[2f27f77d07bc218cc56ee66b11c5f424ec2017ca, f3f...","[api/v1alpha1/common.go, api/v1alpha1/meteor_t...",None,None,61
2,refactor: Move phase const to common.go and re...,- Phases are not `Meteor` resource specific an...,S,tumido,1638474955,1638906644,tumido,1638906644,tumido,1,3,"{'tumido': 1, 'sesheta': 68}","{'821834609': {'author': 'harshad16', 'words_c...","[approved, size/S, lgtm]",[03f3bd708d85a3973d3437c124c2c5ba8023e520],"[api/v1alpha1/common.go, api/v1alpha1/meteor_t...",1638485274,1638485274,60
3,Update bundle prescription,![image](https://user-images.githubusercontent...,XL,tumido,1638289567,1638464194,sesheta,1638464194,sesheta,6,18,"{'pacospace': 39, 'tumido': 20, 'sesheta': 65}",{},"[size/XL, approved, lgtm]","[42bb5be6d3461a2017200e8cb69a8ac53d57318d, 9fc...","[api/v1alpha1/coma_types.go, api/v1alpha1/mete...",None,None,59
4,feat: Rename MeteorComa to Coma,Follow up to #46 \r\n\r\n`s/MeteorComa/Coma/` ...,XL,tumido,1638243344,1638280649,sesheta,1638280649,sesheta,1,19,"{'todo[bot]': 100, 'tumido': 1, 'sesheta': 65}",{},"[size/XL, approved, lgtm]",[434672fd39141098725f5fbe2423ad043704bd44],"[Makefile, PROJECT, api/v1alpha1/coma_types.go...",None,None,54


Let's look at the `Thoth` project repo and fetch its relevant issue/PR data.

In [28]:
thoth_issues = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/thoth-station/core/Issue.json"
)
thoth_prs = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/thoth-station/core/PullRequest.json"
)

Getting dataset from Ceph
Getting dataset from Ceph


In [29]:
thoth_issues.head()

,title,body,created_by,created_at,closed_by,closed_at,labels,interactions,id
0,OCTO Summit Demo,OCTO summit Demo is showcasing an end-to-end d...,goern,1644261684,None,None,"{'needs-triage': {'color': 'ededed', 'labeled_...",{'harshad16': 38},377
1,Bring updated Ray images to Operate-first cloud,## Problem statement\r\nwe would like to provi...,harshad16,1643226449,None,None,"{'kind/feature': {'color': 'c7def8', 'labeled_...",{'goern': 3},372
2,Endpoint for triggering initialization of a re...,## Problem statement\r\n\r\nAs a user of Thoth...,fridex,1643214348,None,None,"{'kind/feature': {'color': 'c7def8', 'labeled_...",{'codificat': 4},371
3,Introduce jupyterlab collaborative,**Is your feature request related to a problem...,pacospace,1642600001,None,None,"{'needs-triage': {'color': 'ededed', 'labeled_...",{'sesheta': 73},383
4,create SBOM for software stack provided,**Is your feature request related to a problem...,goern,1640093888,None,None,"{'kind/feature': {'color': 'c7def8', 'labeled_...","{'goern': 3, 'codificat': 4, 'fridex': 2, 'ses...",366


In [30]:
thoth_prs.head()

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
0,SIG DevSecOps: fold sig-pipelines into it and ...,## Related Issues and Dependencies\r\n<!-- Men...,L,codificat,1644977229,1645034406,sesheta,1645034406,sesheta,2,13,"{'codificat': 3, 'sesheta': 65}","{'884496535': {'author': 'harshad16', 'words_c...","[approved, size/L]","[38a457760409212dba21cf5884b8ecaa780d7f77, 182...","[.pre-commit-config.yaml, .prow.yaml, OWNERS_A...",1645034324,1645034324,382
1,WIP Introduce an ADR for automatic base image ...,## Related Issues and Dependencies\r\nRelated ...,M,mayaCostantini,1644939546,None,None,None,None,1,1,{},{},[],[1df89e14a71211beb580079756725d4893662ed2],[docs/adr/0005-automatically-bump-container-im...,None,None,381
2,Include new repos in thoth-station and remove ...,Include new repos in thoth-station and remove ...,M,harshad16,1644546139,None,None,None,None,1,1,"{'sesheta': 189, 'harshad16': 5, 'codificat': 70}","{'879996941': {'author': 'codificat', 'words_c...",[size/M],[841a5d63b2732d16e63e5ae2cc6cc9de9d1a00da],[community/github-config.yaml],1644593166,None,380
3,Add mi to the subprojects,SSIA,XS,xtuchyna,1644348382,1644432499,sesheta,1644432499,sesheta,3,1,"{'sesheta': 65, 'pacospace': 1, 'xtuchyna': 13}","{'876119118': {'author': 'pacospace', 'words_c...","[approved, size/XS]","[028bb6998d83aa0d1a9d42b81aaa3896e1ae2d06, 9a6...",[community/sig-observability/README.md],1644348764,1644348764,379
4,Include pulp metrics exporter,Signed-off-by: Francesco Murdaca <fmurdaca@red...,XS,pacospace,1644348141,1644350833,sesheta,1644350833,sesheta,1,1,{'sesheta': 65},"{'876154664': {'author': 'harshad16', 'words_c...","[approved, size/XS]",[f5231367fa94262d4ca98b1df4e01ce0cd9b2448],[community/github-config.yaml],1644350008,1644350008,378


Let's look at the `Seraph` project repo and fetch its relevant issue/PR data.

In [31]:
seraph_issues = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/AICoE/seraph/Issue.json"
)
seraph_prs = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/AICoE/seraph/PullRequest.json"
)

Getting dataset from Ceph
Getting dataset from Ceph


In [32]:
seraph_issues.head()

,title,body,created_by,created_at,closed_by,closed_at,labels,interactions,id
0,Provide access to 3scale configuration,We need to be provided read access to repos me...,tumido,1642706468,None,None,{},{},21
1,Deploy Ingress PUPTOO for Advisor,https://github.com/RedHatInsights/insights-pup...,tumido,1642187942,None,None,{},{},20
2,Opensource Advisor,In order to deploy Advisor we need to have the...,tumido,1642187910,None,None,{},{'dkuc': 7},19
3,Deploy Advisor,App Interface:\r\nhttps://visual-app-interface...,tumido,1642187710,None,None,{},{},18
4,[Change bLOG] Seraph,"### Subtitle\r\nSeraph: What was done why, whe...",schwesig,1637929558,None,None,"{'kind/documentation': {'color': 'c7def8', 'la...",{'schwesig': 114},15


In [33]:
seraph_prs.head()

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
0,Add ingress,Resolves: https://github.com/AICoE/seraph/issu...,L,tumido,1642183963,1643122670,sesheta,1643122670,sesheta,2,8,"{'tumido': 1, 'sesheta': 65}",{},"[approved, size/L]","[1661798b20b1cd898f17e11c6e5efe85c3517b43, 850...","[.pre-commit-config.yaml, ingress/base/config....",None,None,17
1,Kafka discovery,For dev purposes let's scale kafka down a bit ...,M,tumido,1641931118,1643122316,sesheta,1643122316,sesheta,4,8,"{'tumido': 1, 'sesheta': 65, 'schwesig': 1}",{},"[approved, size/M]","[35c0d1d7eae33a357bfed9cd3b85c76be4af9bfa, da1...","[.pre-commit-config.yaml, platform-mq/base/kaf...",None,None,16
2,fix: Kafka topics labels,Topics need labels to be associated with Kafka...,S,tumido,1637785666,1639501111,sesheta,1639501111,sesheta,1,7,"{'tumido': 1, 'sesheta': 65}",{},"[size/S, lgtm, approved]",[8513c3a322ecb348bd0c0598abfbdb3842e39ab4],"[platform-mq/base/kustomization.yaml, platform...",None,None,14
3,feat: Init cluster scope resources,An attempt to track cluster scope resources fo...,S,tumido,1637016659,1637094850,sesheta,1637094850,sesheta,1,6,"{'schwesig': 1, 'sesheta': 65}","{'807467120': {'author': 'schwesig', 'words_co...","[size/S, lgtm, approved]",[fada7c1b27006e36f16353567d63e5eff9d41f13],[cluster-scope/base/core/namespaces/ingress/ku...,1637094650,1637094650,13
4,feat: Init Kafka deployment,Resolves: #3 \r\n\r\nInitial insecure Kafka de...,M,tumido,1637016193,1637095496,sesheta,1637095496,sesheta,1,9,"{'sesheta': 65, 'schwesig': 1}","{'807476511': {'author': 'schwesig', 'words_co...","[lgtm, approved, size/M]",[473d657b1a7012814683d8d39b9c972c8bede199],"[platform-mq/base/kafka.yaml, platform-mq/base...",1637095026,1637095026,12


## WG BYON BUild Pipelines
This working group has one sub-project called `BYON`. Lets fetch its relevant issue/PR data from the project repo.

In [34]:
byon_issues = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/open-services-group/byon/Issue.json"
)
byon_prs = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/open-services-group/byon/PullRequest.json"
)

Getting dataset from Ceph
Getting dataset from Ceph


In [35]:
byon_issues.head()

,title,body,created_by,created_at,closed_by,closed_at,labels,interactions,id
0,Define states for Pipelines and UI (e.g. valid...,**Is your feature request related to a problem...,pacospace,1645029887,None,None,{'wg/byon-build-pipelines': {'color': 'd2b48c'...,"{'tumido': 125, 'pacospace': 4}",23
1,Dev/Stage environment for ODH Jupyterhub,**Is your feature request related to a problem...,tumido,1644338006,None,None,{'wg/byon-build-pipelines': {'color': 'd2b48c'...,{'tumido': 225},20
2,Deploy BYON (admin feature) + UI with Open Dat...,**Is your feature request related to a problem...,pacospace,1644242774,None,None,{'wg/byon-build-pipelines': {'color': 'd2b48c'...,{'pacospace': 2},19
3,Define new image tag annotations,Find agreement on new additional imagestream a...,tumido,1643826795,None,None,{'wg/byon-build-pipelines': {'color': 'd2b48c'...,{},18
4,Periodic pooling of container images,Investigate how to manage imagestream updates....,tumido,1643671073,None,None,{'wg/byon-build-pipelines': {'color': 'd2b48c'...,"{'tumido': 766, 'sesheta': 82, 'mimotej': 494}",17


In [36]:
byon_prs.head()

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
0,feat: Quick and dirty dev env setup,## Related Issues and Dependencies\r\n\r\nPart...,L,tumido,1644359830,1644937770,sesheta,1644937770,sesheta,2,9,"{'codificat': 38, 'tumido': 35, 'sesheta': 68}","{'877380995': {'author': 'codificat', 'words_c...","[lgtm, approved, size/L]","[7738746b38bcfb0e79a3bc443d56dac6eedbcc73, 7ee...","[.gitignore, Makefile, README.md, manifests/ba...",1644427690,1644886389,21
1,Update OWNERS,## Related Issues and Dependencies\r\n\r\n…\r\...,XS,pacospace,1642109662,1642109666,pacospace,1642109666,pacospace,1,1,{'sesheta': 180},{},[size/XS],[dfe01ea498301f6687691361915cae9684a03d83],[OWNERS],None,None,9
2,Thoth Configuration Initialization,## Automatic configuration initialization\nThe...,M,khebhut[bot],1642109521,1642109674,pacospace,1642109674,pacospace,1,1,{'sesheta': 205},"{'851956482': {'author': 'pacospace', 'words_c...","[needs-ok-to-test, size/M]",[6e50af51b654e5ef2a99122e3fb5aba46e7a94ba],[.thoth.yaml],1642109598,1642109598,8


## WG DevSecOps

Let's now fetch the relevant issue/PR data for the WG DevSecOps repository.

In [37]:
devsecops_issues = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/open-services-group/devsecops/Issue.json"
)
devsecops_prs = read_data_from_ceph(
    "open-services-group/metrics/github/"
    "srcopsmetrics/bot_knowledge/open-services-group/devsecops/PullRequest.json"
)

Getting dataset from Ceph
Getting dataset from Ceph


In [38]:
devsecops_issues.head()

,title,body,created_by,created_at,closed_by,closed_at,labels,interactions,id
0,What tooling will be first deployed?,After https://github.com/open-services-group/d...,HumairAK,1642215030,None,None,{},{'HumairAK': 16},4
1,What cluster to deploy ODH and associated tool...,We should start discussions on what cluster we...,HumairAK,1642214841,None,None,{},"{'HumairAK': 96, 'durandom': 27}",3
2,Evaluate the system for production ready,As the DevSecOps engineer\r\nI would like to k...,harshad16,1642214122,None,None,{},{},2


In [39]:
devsecops_prs.head()

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
0,adding devsecops base tooling,Pre-commit test ran on #5 but we don't have pr...,M,Gregory-Pereira,1644460128,1644628052,harshad16,1644628052,harshad16,1,3,{'sesheta': 130},"{'879322811': {'author': 'harshad16', 'words_c...","[size/M, do-not-merge/invalid-owners-file]",[b6d32ad8232b3748cf861ea506bf8ad65a457280],"[.pre-commit-config.yaml, .prow.yaml, OWNERS]",1644538174,1644628042,6
1,creating initial structure of contrast adr,"Action item from DevSecOPs WG meeting Feb 9th,...",S,Gregory-Pereira,1644452567,None,None,None,None,2,1,"{'sesheta': 56, 'Gregory-Pereira': 3}",{},[size/S],"[1f35e5fbc6a79462c546d2ae7ff40aea3d0d077c, fe6...",[ODH-RHODS-contrast-ADR.md],None,None,5
2,Thoth Configuration Initialization,## Automatic configuration initialization\nThe...,M,khebhut[bot],1642112594,None,None,None,None,1,1,{'sesheta': 56},{},[size/M],[1afa26af10b156e59393baa17872d1ba0f7c62fc],[.thoth.yaml],None,None,1
